# Data Extraction and Cleaning Step

In [211]:
import pandas as pd
import re
import os

pd.set_option('display.max_colwidth', None)

## Read the txt file and generate a pandas dataframe

In [212]:
# Define Patterns for filtering
image_omitted_message = "‎immagine omessa"
omitted_video_message = "‎video omesso"
one_view_video_message = "‎Hai ricevuto un video visualizzabile una volta. Per garantire una maggiore privacy, potrai aprirlo soltanto sul telefono."
sticker_not_included_message = "‎sticker non incluso"
deleted_nessage = "‎Questo messaggio è stato eliminato."
omitted_audio_message = "‎audio omesso"
one_view_message = "‎Hai inviato un messaggio visualizzabile una volta. Per garantire una maggiore privacy, potrai aprirlo soltanto sul telefono."
excluded_gif_message = "‎GIF esclusa"
edited_message = "‎<Questo messaggio è stato modificato>"
error_message = "‎In attesa del messaggio. Potrebbe volerci un po' di tempo."
one_view_image = "Hai ricevuto una foto visualizzabile una volta. Per garantire una maggiore privacy, potrai aprirla soltanto sul telefono."
you_admin_deleted_message = "‎Hai eliminato questo messaggio come amministratore."
admin_deleted_message = "‎Questo messaggio è stato eliminato dall'amministratore"
one_listen_audio = "‎Hai ricevuto un messaggio vocale riproducibile una volta."
is_pdf = ".pdf"
missed_call = "‎Videochiamata persa"
url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [ ]:
filtered_lines = []

with open("../data/full_chat.txt", "r", encoding="utf-8") as input_file:
    lines = input_file.readlines()
    
for line in lines:
    if ("cccccccc: " in line and # masked the sender name for privacy concerns
        image_omitted_message not in line and
        sticker_not_included_message not in line and
        deleted_nessage not in line and 
        omitted_audio_message not in line and 
        excluded_gif_message not in line and
        one_view_message not in line and
        one_view_image not in line and
        one_view_video_message not in line and 
        omitted_video_message not in line and
        you_admin_deleted_message not in line and
        admin_deleted_message not in line and
        error_message not in line and
        is_pdf not in line and
        missed_call not in line and
        one_listen_audio not in line and
        not re.search(url_pattern, line)
        ):
        line = line.replace(edited_message, "").strip()
        filtered_lines.append(line)

pattern = r'\[(\d{2}/\d{2}/\d{2}, \d{2}:\d{2}:\d{2})\] (.*?): (.*?)(?=\n\[\d{2}/\d{2}/\d{2}, \d{2}:\d{2}:\d{2}\]|\Z)'
content = '\n'.join(filtered_lines)
messages = re.findall(pattern, content, re.DOTALL)

df = pd.DataFrame(messages, columns=['timestamp', 'sender', 'message'])
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%d/%m/%y, %H:%M:%S')

#df

,timestamp,sender,message
0,2024-05-16 18:42:48,Carmine,Ma dalli a sheila
1,2024-05-16 18:42:56,Carmine,Tanto è abituata
2,2024-05-16 18:43:01,Carmine,Oppure sotto qualche ruota
3,2024-05-16 18:43:04,Carmine,Ma vedi che nn ti giudico…
4,2024-05-16 18:43:58,Carmine,Capita un po spesso…
...,...,...,...
80870,2025-04-12 15:28:26,Carmine,Capisci
80871,2025-04-12 15:28:29,Carmine,Mi hai chiesto lego marvel
80872,2025-04-12 15:28:31,Carmine,Cosa ti posso mai dire
80873,2025-04-12 15:28:35,Carmine,Si gioca a 2 bro…


## Export Text Sequence for Custom Tokenizer

In [ ]:
text_sequence = " ".join(df['message'].values)

len(text_sequence)

output_path = "../output/messages.txt"

with open(output_path, "w") as f:
    f.write(text_sequence)